In [ ]:
from langchain import OpenAI
llm = OpenAI(temperature=0)

In [ ]:
from CQLDatabaseChain import CQLDatabase, CQLDatabaseChain

In [ ]:
from cqlsession import getCqlSession

In [ ]:
astraSession = getCqlSession()

In [ ]:
db = CQLDatabase(
    astraSession,
    'pqdemo',
#     ignore_tables = ['base_types', 'coll_test', 'w_counter'],
#     include_tables = ['people', 'w_counter'],
)

In [ ]:
db_chain = CQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
# db_chain.run("How many people live in milan?")
# db_chain.run("Are there people living in milan ?")
db_chain.run("Are there folks living in barcelona ?")
# db_chain.run("What is the average temperature on Saturn ?")

In [ ]:
###

In [ ]:
from typing import Any, Dict, List, Optional

from langchain.chains.llm import LLMChain
from langchain.chains.base import Chain
from langchain.chains.sql_database.prompt import DECIDER_PROMPT
from langchain.base_language import BaseLanguageModel
from langchain.prompts.base import BasePromptTemplate
from langchain.chains import SQLDatabaseSequentialChain
#
from CQLDatabaseChain import CQL_PROMPT


class CQLDatabaseSequentialChain(SQLDatabaseSequentialChain):
    """CQL variant."""

    decider_chain: LLMChain
    sql_chain: CQLDatabaseChain
    input_key: str = "query"  #: :meta private:
    output_key: str = "result"  #: :meta private:
    return_intermediate_steps: bool = False

    @classmethod
    def from_llm(
        cls,
        llm: BaseLanguageModel,
        database: CQLDatabase,
        query_prompt: BasePromptTemplate = CQL_PROMPT,
        decider_prompt: BasePromptTemplate = DECIDER_PROMPT,
        **kwargs: Any,
    ):
        """Load the necessary chains."""
        sql_chain = CQLDatabaseChain(
            llm=llm, database=database, prompt=query_prompt, **kwargs
        )
        decider_chain = LLMChain(
            llm=llm, prompt=decider_prompt, output_key="table_names"
        )
        return cls(sql_chain=sql_chain, decider_chain=decider_chain, **kwargs)


In [ ]:
chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)

In [ ]:
chain.run("How many people live in madrid ?")